# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」コンペティションにおいて、チャットボットの応答がどちらのモデルに好まれるかを予測する問題に取り組んでいます。具体的には、異なる大規模言語モデル（LLM）による応答の比較を行い、どちらの応答がユーザーに受け入れられるかを予測するための機械学習モデルを構築しています。

### 問題へのアプローチ
1. **データ準備**:
   - テストデータとサンプル提出ファイルをCSVから読み込み、必要なデータを整形します。
   - ユーザープロンプトおよびモデルの応答を連結し、モデルの入力形式に適したテキストを準備します。

2. **ライブラリの使用**:
   - `torch` (深層学習フレームワーク) と `sklearn` (機械学習ツール) のほか、`transformers`ライブラリを利用して、LLMのベースモデル（Llama）を読み込んでいます。
   - `peft` (Parameter-Efficient Fine-Tuning)ライブラリを用いてモデルを効率的にファインチューニングします。

3. **トークナイズ**:
   - `AutoTokenizer`を使用して、モデルへの入力データをトークン化し、適切な形状にパディングします。

4. **モデルの構築と推論**:
   - Chatbot Arenaでの競争用に、大規模なLLMをモデルとして設定し、複数のGPUを用いて並列処理を行います。
   - `LoraConfig`を通じてLoRA（Low-Rank Adaptation）によるパラメータの効率的な微調整を行います。
   - 推論関数を定義し、入力データに対してモデルの出力を早く計算します。

5. **LightGBMの使用**:
   - LightGBMを用いた新たな特徴量抽出と分類モデルによって、応答の選好予測も行います。
   - 以前に保存されたモデルを用いて推論を実施し、各応答のクラス確率を出力します。

6. **予測のブレンド**:
   - 最後に、LGBMモデルトとLLMモデルの予測を重み付けしてブレンドし、最終的な予測結果を生成します。

### 使用ライブラリ
- `torch`: 深層学習モデルのライブラリ。
- `transformers`: LLMの読み込みやトークナイズ用ライブラリ。
- `peft`: モデルのファインチューニングを効率化するためのライブラリ。
- `pandas`: データ処理ライブラリ。
- `sklearn`: 機械学習用ライブラリ（モデル評価、データ処理）。
- `lightgbm`: 勾配ブースティングフレームワーク。
- `matplotlib`: グラフ描画ライブラリ（必要に応じて使用）。

このノートブックは、最新の機械学習手法を用いて、ユーザーが最も好むチャットボットの応答を的確に予測するモデルを構築するプロセスを詳細に示しています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に基づいて、機械学習・深層学習の初心者がつまずく可能性がある専門用語の簡単な解説です。

1. **bitsandbytes**: これは、モデルのメモリ使用量を削減し、効率的な計算を可能にするためのライブラリで、量子化技術を利用してモデルを軽量化します。特に、深層学習フレームワークで大規模なモデルを扱う際に役立ちます。

2. **PEFT (Parameter-Efficient Fine-Tuning)**: 自然言語処理モデルを効率的に微調整する技術の一つです。PEFTでは、モデル全体をトレーニングするのではなく、軽微なパラメータ（例: LoRA）だけを学習させることで、計算リソースを節約しながら性能を向上させるアプローチです。

3. **LoRa (Low-Rank Adaptation)**: PEFTの一種で、モデルの重みを低ランクの行列で approximating し、さらにドロップアウトを加えることで、特定のタスクに適したモデルに微調整します。この方法は、学習を早め、少ないデータでの性能向上を目指します。

4. **autocast**: 自動混合精度を使用して、計算の精度を動的に調整するための機能です。高精度の計算を必要としない部分では、計算を16ビット浮動小数点（float16）で行うことで、メモリ使用量を減らし、速度を向上させることができます。

5. **attention mask**: トランスフォーマーモデルにおける入力データの一部を制御するためのマスクです。通常、パディングトークンの部分には0、実際のデータの部分には1が割り当てられ、モデルが無関係な情報を無視するのに使われます。

6. **softmax**: 出力の値を0〜1の範囲に変換し、合計が1になるように正規化する関数です。主に多クラス分類問題で使用され、各クラスが選ばれる確率を示します。

7. **ガベージコレクション (Garbage Collection)**: 使用されなくなったメモリを自動的に解放するプロセスです。Pythonの動的メモリ管理機能の一部で、メモリリークを防ぐために重要です。

8. **シンメトリック対数変換 (Symmetric Log Transformation)**: 数値のスケーリングに使用される手法で、ゼロまたは負の値を持つ場合でも適用できるように、数値の符号を考慮して対数変換を行います。データの分布を正規分布に近づけるためによく使われます。

9. **CountVectorizer**: 文書中の単語の出現回数をカウントし、特長をベクトル形式で表現するための手法です。自然言語処理で一般的に使用され、テキストデータを数値データに変換する際に役立ちます。

10. **StratifiedKFold**: 機械学習の交差検証手法の一つで、各フォールドにおいてクラスの分布が訓練データ全体と均等になるようにデータを分割します。これにより、特に不均衡なデータセットでのモデル評価がより信頼性のあるものになります。

これらの用語は、それぞれ特定の技術やアプローチを示しており、初心者が理解する上での難所となることが多いです。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
<div class="column-right">

# 日本語訳

```python
# bitsandbytesを最新バージョンにアップグレードしてインストールします。
# インデックスを使用せず、指定したリンクからパッケージを探します。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/

# transformersライブラリを最新バージョンにアップグレードしてインストールします。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/

# tokenizersライブラリを最新バージョンにアップグレードしてインストールします。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/

# peftライブラリを最新バージョンにアップグレードしてインストールします。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
</details>

In [ ]:
# bitsandbytesを最新バージョンにアップグレードしてインストールします。
# インデックスを使用せず、指定したリンクからパッケージを探します。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/

# transformersライブラリを最新バージョンにアップグレードしてインストールします。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/

# tokenizersライブラリを最新バージョンにアップグレードしてインストールします。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/

# peftライブラリを最新バージョンにアップグレードしてインストールします。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The work in this notebook is inspired by these notebooks:
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

## Importing Libraries

</div>
<div class="column-right">

# 日本語訳

このノートブックの作業は、以下のノートブックにインスパイアされています：
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

## ライブラリのインポート

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from threading import Thread

import gc
import os
import io
import time
import json
import random
import pickle
import zipfile
import datetime
import matplotlib.pyplot as plt
from IPython.display import display
from collections import Counter
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import tokenizers


```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインポートします。
import torch  # PyTorchをインポートして、深層学習モデルの構築に使用します。
import sklearn  # 機械学習のためのツールを提供するライブラリ
import numpy as np  # 数値計算を行うためのライブラリ
import pandas as pd  # データ操作と解析のためのライブラリ
import time  # 時間に関連する機能を扱うライブラリ

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# Transformersライブラリからトークナイザーやモデルをインポートします。

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# PEFT（Parameter-Efficient Fine-Tuning）ライブラリから必要なクラスや関数をインポートします。

from torch.cuda.amp import autocast  # 自動混合精度トレーニングのための機能をインポートします。
from threading import Thread  # スレッド処理のためのライブラリをインポートします。

import gc  # ガベージコレクションを管理するためのライブラリ
import os  # オペレーティングシステムの機能を操作するライブラリ
import io  # 入出力関連の操作を行うライブラリ
import time  # 時間操作のためのライブラリ
import json  # JSONデータの操作を行うためのライブラリ
import random  # 乱数生成のためのライブラリ
import pickle  # オブジェクトのシリアライズを行うためのライブラリ
import zipfile  # ZIPアーカイブの読み書きを扱うライブラリ
import datetime  # 日付と時間の操作を行うためのライブラリ
import matplotlib.pyplot as plt  # グラフ作成のためのライブラリ
from IPython.display import display  # Jupyter Notebookでの表示を管理するための機能をインポートします。
from collections import Counter  # 頻度計算を行うためのクラス
from collections import defaultdict  # デフォルト値の付いた辞書を作成するためのクラス
import torch  # もう一度PyTorchをインポートします。
from torch import nn  # PyTorchのニューラルネットワーク関連モジュールをインポートします。
import torch.nn.functional as F  # ニューラルネットワークの機能的な操作を提供します。
import pytorch_lightning as pl  # PyTorchを使った機械学習モデルの訓練を簡素化するライブラリ
from torch.utils.data import Dataset, DataLoader  # データセットとデータローダーのクラスをインポートします。
from sklearn.metrics import log_loss  # ログ損失を計算するための関数をインポートします。
import tokenizers  # トークナイザー関連の機能を提供するライブラリ
```

</div>
</details>

In [ ]:
# 必要なライブラリをインポートします。
import torch  # PyTorchをインポートして、深層学習モデルの構築に使用します。
import sklearn  # 機械学習のためのツールを提供するライブラリ
import numpy as np  # 数値計算を行うためのライブラリ
import pandas as pd  # データ操作と解析のためのライブラリ
import time  # 時間に関連する機能を扱うライブラリ

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# Transformersライブラリからトークナイザーやモデルをインポートします。

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# PEFT（Parameter-Efficient Fine-Tuning）ライブラリから必要なクラスや関数をインポートします。

from torch.cuda.amp import autocast  # 自動混合精度トレーニングのための機能をインポートします。
from threading import Thread  # スレッド処理のためのライブラリをインポートします。

import gc  # ガベージコレクションを管理するためのライブラリ
import os  # オペレーティングシステムの機能を操作するライブラリ
import io  # 入出力関連の操作を行うライブラリ
import time  # 時間操作のためのライブラリ
import json  # JSONデータの操作を行うためのライブラリ
import random  # 乱数生成のためのライブラリ
import pickle  # オブジェクトのシリアライズを行うためのライブラリ
import zipfile  # ZIPアーカイブの読み書きを扱うライブラリ
import datetime  # 日付と時間の操作を行うためのライブラリ
import matplotlib.pyplot as plt  # グラフ作成のためのライブラリ
from IPython.display import display  # Jupyter Notebookでの表示を管理するための機能をインポートします。
from collections import Counter  # 頻度計算を行うためのクラス
from collections import defaultdict  # デフォルト値の付いた辞書を作成するためのクラス
import torch  # もう一度PyTorchをインポートします。
from torch import nn  # PyTorchのニューラルネットワーク関連モジュールをインポートします。
import torch.nn.functional as F  # ニューラルネットワークの機能的な操作を提供します。
import pytorch_lightning as pl  # PyTorchを使った機械学習モデルの訓練を簡素化するライブラリ
from torch.utils.data import Dataset, DataLoader  # データセットとデータローダーのクラスをインポートします。
from sklearn.metrics import log_loss  # ログ損失を計算するための関数をインポートします。
import tokenizers  # トークナイザー関連の機能を提供するライブラリ

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
MAX_LENGTH = 1284
BATCH_SIZE = 8
DEVICE = torch.device("cuda")    
```

</div>
<div class="column-right">

# 日本語訳

```python
# メモリ効率の良い分散処理およびフラッシュ分散処理を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# GPUが利用可能であるか確認し、利用できない場合はメッセージを表示します。
if (not torch.cuda.is_available()): 
    print("申し訳ありません - GPUが必要です！")

# モデル名、重みのパス、最大シーケンス長、バッチサイズ、デバイスの設定を定義します。
MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルのディレクトリ
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'  # モデルの重みが保存されているパス
MAX_LENGTH = 1284  # 最大シーケンス長を1284に設定
BATCH_SIZE = 8  # 訓練時に使用するバッチサイズを8に設定
DEVICE = torch.device("cuda")  # 使用するデバイスとしてGPUを指定
```

</div>
</details>

In [ ]:
# メモリ効率の良い分散処理およびフラッシュ分散処理を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# GPUが利用可能であるか確認し、利用できない場合はメッセージを表示します。
if (not torch.cuda.is_available()): 
    print("申し訳ありません - GPUが必要です！")

# モデル名、重みのパス、最大シーケンス長、バッチサイズ、デバイスの設定を定義します。
MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルのディレクトリ
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'  # モデルの重みが保存されているパス
MAX_LENGTH = 1284  # 最大シーケンス長を1284に設定
BATCH_SIZE = 8  # 訓練時に使用するバッチサイズを8に設定
DEVICE = torch.device("cuda")  # 使用するデバイスとしてGPUを指定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Prepare Data 

</div>
<div class="column-right">

# 日本語訳

## データの準備

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータをCSVファイルから読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータのデータフレームを作成
# サンプルの提出ファイルをCSVファイルから読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # 提出サンプルデータのデータフレームを作成
```

</div>
</details>

In [ ]:
# テストデータをCSVファイルから読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータのデータフレームを作成
# サンプルの提出ファイルをCSVファイルから読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # 提出サンプルデータのデータフレームを作成

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# concatenate strings in list
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

display(sample_sub)
display(test.head(5))

# Prepare text for model
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + test['response_b']
print(test['text'][0])
```

</div>
<div class="column-right">

# 日本語訳

```python
# 文字列のリストを連結するための関数を定義します。
def process(input_str):
    stripped_str = input_str.strip('[]')  # 最初と最後の角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文を処理し、余分な引用符を取り除く
    return ' '.join(sentences)  # 文を空白で結合して返す

# テストデータの各カラムに対してprocess関数を適用します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)  # 'prompt'カラムを処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # 'response_a'カラムを処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # 'response_b'カラムを処理

# サンプル提出データとテストデータの先頭5行を表示します。
display(sample_sub)  # サンプル提出データを表示
display(test.head(5))  # 処理後のテストデータの最初の5行を表示

# モデル用にテキストを準備します。
test['text'] = 'User prompt: ' + test['prompt'] + '\n\nModel A :\n' + test['response_a'] + '\n\n--------\n\nModel B:\n' + test['response_b']
# 処理されたテキストの最初の要素を出力します。
print(test['text'][0])  # 最初のテキストを表示
```

</div>
</details>

In [ ]:
# 文字列のリストを連結するための関数を定義します。
def process(input_str):
    stripped_str = input_str.strip('[]')  # 最初と最後の角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文を処理し、余分な引用符を取り除く
    return ' '.join(sentences)  # 文を空白で結合して返す

# テストデータの各カラムに対してprocess関数を適用します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)  # 'prompt'カラムを処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # 'response_a'カラムを処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # 'response_b'カラムを処理

# サンプル提出データとテストデータの先頭5行を表示します。
display(sample_sub)  # サンプル提出データを表示
display(test.head(5))  # 処理後のテストデータの最初の5行を表示

# モデル用にテキストを準備します。
test['text'] = 'User prompt: ' + test['prompt'] + '\n\nModel A :\n' + test['response_a'] + '\n\n--------\n\nModel B:\n' + test['response_b']
# 処理されたテキストの最初の要素を出力します。
print(test['text'][0])  # 最初のテキストを表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Tokenize

</div>
<div class="column-right">

# 日本語訳

## トークナイズ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# Move tensors to CPU and convert them to lists
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
data[:2]
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーを指定したパスから読み込みます。
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストデータをトークン化し、最大長さでパディングします。
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# トークンの入力IDとアテンションマスクをデバイスに移動し、データ型をint32に設定します。
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動し、リストに変換します。
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]  # 入力IDをリストに変換
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]  # アテンションマスクをリストに変換

# 新しいデータフレームを作成し、INPUT_IDSとATTENTION_MASKSカラムを追加します。
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu  # 入力IDをDataFrameに追加
data['ATTENTION_MASKS'] = attention_masks_cpu  # アテンションマスクをDataFrameに追加

# 最初の2行を表示します。
data[:2]  # データフレームの最初の2行を表示
```

</div>
</details>

In [ ]:
# トークナイザーを指定したパスから読み込みます。
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストデータをトークン化し、最大長さでパディングします。
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# トークンの入力IDとアテンションマスクをデバイスに移動し、データ型をint32に設定します。
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動し、リストに変換します。
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]  # 入力IDをリストに変換
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]  # アテンションマスクをリストに変換

# 新しいデータフレームを作成し、INPUT_IDSとATTENTION_MASKSカラムを追加します。
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu  # 入力IDをDataFrameに追加
data['ATTENTION_MASKS'] = attention_masks_cpu  # アテンションマスクをDataFrameに追加

# 最初の2行を表示します。
data[:2]  # データフレームの最初の2行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load model 
> We load 1 model on each gpu.  

</div>
<div class="column-right">

# 日本語訳

## モデルの読み込み
> 各GPUに1つのモデルを読み込みます。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# BitsAndBytes configuration
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False)

# Load base model on GPU 0
device0 = torch.device('cuda:0')

base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:0')
base_model_0.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# BitsAndBytesの設定を行います。
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでモデルを読み込む設定
    bnb_8bit_compute_dtype=torch.float16,  # 8ビット計算のデータ型をfloat16に設定
    bnb_8bit_use_double_quant=False)  # ダブル量子化を使用しない設定

# GPU 0にベースモデルを読み込みます。
device0 = torch.device('cuda:0')

# LlamaForSequenceClassificationモデルを指定した設定で読み込みます。
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類タスクのラベル数を3に設定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化の設定を適用
    device_map='cuda:0')  # モデルをGPU 0に割り当て

# パディングトークンのIDをトークナイザーから取得し、モデルの設定に適用します。
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンのIDを設定
```

</div>
</details>

In [ ]:
# BitsAndBytesの設定を行います。
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでモデルを読み込む設定
    bnb_8bit_compute_dtype=torch.float16,  # 8ビット計算のデータ型をfloat16に設定
    bnb_8bit_use_double_quant=False)  # ダブル量子化を使用しない設定

# GPU 0にベースモデルを読み込みます。
device0 = torch.device('cuda:0')

# LlamaForSequenceClassificationモデルを指定した設定で読み込みます。
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類タスクのラベル数を3に設定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化の設定を適用
    device_map='cuda:0')  # モデルをGPU 0に割り当て

# パディングトークンのIDをトークナイザーから取得し、モデルの設定に適用します。
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンのIDを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load base model on GPU 1
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
base_model_1.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU 1にベースモデルを読み込みます。
device1 = torch.device('cuda:1')

# LlamaForSequenceClassificationモデルを指定した設定で読み込みます。
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類タスクのラベル数を3に設定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化の設定を適用
    device_map='cuda:1')  # モデルをGPU 1に割り当て

# パディングトークンのIDをトークナイザーから取得し、モデルの設定に適用します。
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンのIDを設定
```

</div>
</details>

In [ ]:
# GPU 1にベースモデルを読み込みます。
device1 = torch.device('cuda:1')

# LlamaForSequenceClassificationモデルを指定した設定で読み込みます。
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類タスクのラベル数を3に設定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化の設定を適用
    device_map='cuda:1')  # モデルをGPU 1に割り当て

# パディングトークンのIDをトークナイザーから取得し、モデルの設定に適用します。
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンのIDを設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load weights 


</div>
<div class="column-right">

# 日本語訳

## 重みの読み込み

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# LoRa configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj'])
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoRaの設定を行います。
peft_config = LoraConfig(
    r=16,  # LoRaのランク
    lora_alpha=32,  # LoRaのスケーリング係数
    lora_dropout=0.10,  # ドロップアウト率
    bias='none',  # バイアスの使用設定
    inference_mode=True,  # 推論モードの設定
    task_type=TaskType.SEQ_CLS,  # タスクの種類をシーケンス分類に設定
    target_modules=['o_proj', 'v_proj']  # 対象とするモジュール
)
```

</div>
</details>

In [ ]:
# LoRaの設定を行います。
peft_config = LoraConfig(
    r=16,  # LoRaのランク
    lora_alpha=32,  # LoRaのスケーリング係数
    lora_dropout=0.10,  # ドロップアウト率
    bias='none',  # バイアスの使用設定
    inference_mode=True,  # 推論モードの設定
    task_type=TaskType.SEQ_CLS,  # タスクの種類をシーケンス分類に設定
    target_modules=['o_proj', 'v_proj']  # 対象とするモジュール
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Get peft
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
#Load weights
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_0.eval()

model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_1.eval()

#Trainable Parameters
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFTモデルを取得し、GPU 0に移動します。
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
# 重みを読み込みます。
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # モデルに重みを適用
model_0.eval()  # 評価モードに設定

# GPU 1に対して同様の処理を行います。
model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # モデルに重みを適用
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータを表示します。
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()  # 各モデルの学習可能なパラメータを出力
```

</div>
</details>

In [ ]:
# PEFTモデルを取得し、GPU 0に移動します。
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
# 重みを読み込みます。
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # モデルに重みを適用
model_0.eval()  # 評価モードに設定

# GPU 1に対して同様の処理を行います。
model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # モデルに重みを適用
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータを表示します。
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()  # 各モデルの学習可能なパラメータを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gc.collect()
```

</div>
<div class="column-right">

# 日本語訳

```python
# ガベージコレクションを実行して、メモリを解放します。
gc.collect()  # 未使用のメモリを整理して、システムの効率を向上させるために呼び出します。
```

</div>
</details>

In [ ]:
# ガベージコレクションを実行して、メモリを解放します。
gc.collect()  # 未使用のメモリを整理して、システムの効率を向上させるために呼び出します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Inference

</div>
<div class="column-right">

# 日本語訳

## 推論

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        generated_class_a.extend(probabilities[:, 0])
        generated_class_b.extend(probabilities[:, 1])
        generated_class_c.extend(probabilities[:, 2])
    
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  

    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を行う関数を定義します。
def inference(df, model, device, batch_size=BATCH_SIZE):
    # DataFrameから入力IDとアテンションマスクをテンソルとして取得します。
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    # 各クラスの生成結果を保持するリストを初期化します。
    generated_class_a = []  # モデルAの結果を格納
    generated_class_b = []  # モデルBの結果を格納
    generated_class_c = []  # 引き分けの結果を格納

    model.eval()  # モデルを評価モードに設定
    
    # バッチ処理でデータを処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに転送
        
        with torch.no_grad():  # 勾配計算を無効にします。
            with autocast():  # 自動混合精度を使用して計算を行います。
                outputs = model(
                    input_ids=batch_input_ids,  # バッチの入力IDをモデルに渡す
                    attention_mask=batch_attention_mask  # バッチのアテンションマスクをモデルに渡す
                )
        
        # モデルの出力をソフトマックス関数を用いて確率に変換します。
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        # 各クラスの確率をリストに追加します。
        generated_class_a.extend(probabilities[:, 0])  # クラスAの確率を拡張
        generated_class_b.extend(probabilities[:, 1])  # クラスBの確率を拡張
        generated_class_c.extend(probabilities[:, 2])  # 引き分けの確率を拡張
    
    # DataFrameに生成した結果を追加します。
    df['winner_model_a'] = generated_class_a  # モデルAの勝者確率を追加
    df['winner_model_b'] = generated_class_b  # モデルBの勝者確率を追加
    df['winner_tie'] = generated_class_c  # 引き分け確率を追加

    torch.cuda.empty_cache()  # GPUのキャッシュをクリアしてメモリを解放

    return df  # 結果を含むDataFrameを返します。
```

</div>
</details>

In [ ]:
# 推論を行う関数を定義します。
def inference(df, model, device, batch_size=BATCH_SIZE):
    # DataFrameから入力IDとアテンションマスクをテンソルとして取得します。
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    # 各クラスの生成結果を保持するリストを初期化します。
    generated_class_a = []  # モデルAの結果を格納
    generated_class_b = []  # モデルBの結果を格納
    generated_class_c = []  # 引き分けの結果を格納

    model.eval()  # モデルを評価モードに設定
    
    # バッチ処理でデータを処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに転送
        
        with torch.no_grad():  # 勾配計算を無効にします。
            with autocast():  # 自動混合精度を使用して計算を行います。
                outputs = model(
                    input_ids=batch_input_ids,  # バッチの入力IDをモデルに渡す
                    attention_mask=batch_attention_mask  # バッチのアテンションマスクをモデルに渡す
                )
        
        # モデルの出力をソフトマックス関数を用いて確率に変換します。
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        # 各クラスの確率をリストに追加します。
        generated_class_a.extend(probabilities[:, 0])  # クラスAの確率を拡張
        generated_class_b.extend(probabilities[:, 1])  # クラスBの確率を拡張
        generated_class_c.extend(probabilities[:, 2])  # 引き分けの確率を拡張
    
    # DataFrameに生成した結果を追加します。
    df['winner_model_a'] = generated_class_a  # モデルAの勝者確率を追加
    df['winner_model_b'] = generated_class_b  # モデルBの勝者確率を追加
    df['winner_tie'] = generated_class_c  # 引き分け確率を追加

    torch.cuda.empty_cache()  # GPUのキャッシュをクリアしてメモリを解放

    return df  # 結果を含むDataFrameを返します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

N_SAMPLES = len(data)

# Split the data into two subsets
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

# Function to run inference in a thread
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)

# Dictionary to store results from threads
results = {}
```

</div>
<div class="column-right">

# 日本語訳

```python
# 時間計測を開始します。
st = time.time()

# データのサンプル数を取得します。
N_SAMPLES = len(data)

# データを2つのサブセットに分割します。
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算
sub1 = data.iloc[0:half].copy()  # 最初の半分のデータをコピー
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りのデータをコピー

# スレッドで推論を実行するための関数を定義します。
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 指定したインデックスに推論結果を保存

# スレッドからの結果を格納するための辞書を初期化します。
results = {}  # 結果を格納する空の辞書を作成
```

</div>
</details>

In [ ]:
# 時間計測を開始します。
st = time.time()

# データのサンプル数を取得します。
N_SAMPLES = len(data)

# データを2つのサブセットに分割します。
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算
sub1 = data.iloc[0:half].copy()  # 最初の半分のデータをコピー
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りのデータをコピー

# スレッドで推論を実行するための関数を定義します。
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 指定したインデックスに推論結果を保存

# スレッドからの結果を格納するための辞書を初期化します。
results = {}  # 結果を格納する空の辞書を作成

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# start threads
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

# Wait for all threads to finish
t0.join()
t1.join()

# Combine results back into the original DataFrame
data = pd.concat([results[0], results[1]], axis=0)

print(f"Processing complete. Total time: {time.time() - st}")

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = data[TARGETS]
```

</div>
<div class="column-right">

# 日本語訳

```python
# スレッドを開始します。
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # スレッド0を初期化
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # スレッド1を初期化

t0.start()  # スレッド0を開始
t1.start()  # スレッド1を開始

# すべてのスレッドが終了するのを待ちます。
t0.join()  # スレッド0の終了を待機
t1.join()  # スレッド1の終了を待機

# 結果を元のDataFrameに結合します。
data = pd.concat([results[0], results[1]], axis=0)  # 2つの結果を縦に結合

# 処理が完了したことを表示し、所要時間を出力します。
print(f"処理が完了しました。総時間: {time.time() - st}")

# ターゲットとするカラムをリスト化します。
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

# サンプル提出データに推論結果を追加します。
sample_sub[TARGETS] = data[TARGETS]  # ターゲットの結果を提出サンプルに追加
```

</div>
</details>

In [ ]:
# スレッドを開始します。
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # スレッド0を初期化
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # スレッド1を初期化

t0.start()  # スレッド0を開始
t1.start()  # スレッド1を開始

# すべてのスレッドが終了するのを待ちます。
t0.join()  # スレッド0の終了を待機
t1.join()  # スレッド1の終了を待機

# 結果を元のDataFrameに結合します。
data = pd.concat([results[0], results[1]], axis=0)  # 2つの結果を縦に結合

# 処理が完了したことを表示し、所要時間を出力します。
print(f"処理が完了しました。総時間: {time.time() - st}")

# ターゲットとするカラムをリスト化します。
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

# サンプル提出データに推論結果を追加します。
sample_sub[TARGETS] = data[TARGETS]  # ターゲットの結果を提出サンプルに追加

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
llama_preds = data[TARGETS].values
```

</div>
<div class="column-right">

# 日本語訳

```python
# ターゲットカラムから予測結果を抽出し、NumPy配列に変換します。
llama_preds = data[TARGETS].values  # 推論結果をNumPy配列として取得
```

</div>
</details>

In [ ]:
# ターゲットカラムから予測結果を抽出し、NumPy配列に変換します。
llama_preds = data[TARGETS].values  # 推論結果をNumPy配列として取得

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## LGBM + tfidf

</div>
<div class="column-right">

# 日本語訳

## LGBM + TF-IDF

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TAG = 'lmsys-chatbot-arena'

import os
RUNPOD = os.path.exists('/workspace/')
KAGGLE = not RUNPOD
if KAGGLE: print('kaggle')
```

</div>
<div class="column-right">

# 日本語訳

```python
# コンペティションのタグを設定します。
TAG = 'lmsys-chatbot-arena'

import os  # osモジュールをインポート

# 実行環境がRUNPODかどうかをチェックします。
RUNPOD = os.path.exists('/workspace/')  # '/workspace/'が存在するか確認
KAGGLE = not RUNPOD  # RUNPODでない場合はKAGGLE環境とする
if KAGGLE: 
    print('kaggle')  # KAGGLE環境であればメッセージを表示
```

</div>
</details>

In [ ]:
# コンペティションのタグを設定します。
TAG = 'lmsys-chatbot-arena'

import os  # osモジュールをインポート

# 実行環境がRUNPODかどうかをチェックします。
RUNPOD = os.path.exists('/workspace/')  # '/workspace/'が存在するか確認
KAGGLE = not RUNPOD  # RUNPODでない場合はKAGGLE環境とする
if KAGGLE: 
    print('kaggle')  # KAGGLE環境であればメッセージを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
try:
    import pandas as pd
except:
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba
```

</div>
<div class="column-right">

# 日本語訳

```python
# pandasをインポートしてみます。
try:
    import pandas as pd  # pandasをインポート
except:  # インポートに失敗した場合
    # Kaggle APIと必要なライブラリをインストールします。
    !pip install -q kaggle  # Kaggle APIをインストール
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # データ処理ライブラリをインストール
    !pip install -q protobuf  # protobufライブラリをインストール
    !pip install -q numba  # numbaライブラリをインストール
```

</div>
</details>

In [ ]:
# pandasをインポートしてみます。
try:
    import pandas as pd  # pandasをインポート
except:  # インポートに失敗した場合
    # Kaggle APIと必要なライブラリをインストールします。
    !pip install -q kaggle  # Kaggle APIをインストール
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # データ処理ライブラリをインストール
    !pip install -q protobuf  # protobufライブラリをインストール
    !pip install -q numba  # numbaライブラリをインストール

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

import os

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json
        !chmod 600 /root/.kaggle/kaggle.json

    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/ 
        
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')    
```

</div>
<div class="column-right">

# 日本語訳

```python
# データのパスを設定します。
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)  # 環境に応じてデータのパスを決定

import os  # osモジュールを再度インポートします。

# 実行環境がRUNPODの場合の処理
if RUNPOD:
    # Kaggle認証ファイルが存在しない場合
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # .kaggleディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # kaggle.jsonをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # 権限を設定
    
    # コンペティションのZIPファイルが存在しない場合
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # Kaggleからデータをダウンロード
        
    # データを格納するディレクトリが存在しない場合
    if not os.path.exists('/data/'):
        import zipfile  # zipファイル処理用のライブラリをインポート
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍してデータを/data/に展開
```

</div>
</details>

In [ ]:
# データのパスを設定します。
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)  # 環境に応じてデータのパスを決定

import os  # osモジュールを再度インポートします。

# 実行環境がRUNPODの場合の処理
if RUNPOD:
    # Kaggle認証ファイルが存在しない場合
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # .kaggleディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # kaggle.jsonをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # 権限を設定
    
    # コンペティションのZIPファイルが存在しない場合
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # Kaggleからデータをダウンロード
        
    # データを格納するディレクトリが存在しない場合
    if not os.path.exists('/data/'):
        import zipfile  # zipファイル処理用のライブラリをインポート
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍してデータを/data/に展開

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
# MODEL_PATH = MODEL_PATH if not KAGGLE else ''#MODEL_PATH + os.listdir(MODEL_PATH)[0] + '/'
print(MODEL_PATH)

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''
```

</div>
<div class="column-right">

# 日本語訳

```python
# 入力、モデル、ログ用のパスを設定します。
INPUT_PATH = '/kaggle/input/'  # Kaggleの入力データパス
MODEL_PATH = '/workspace/models/'  # モデル保存用のパス
LOGITS_PATH = '/workspace/logits/'  # ログ保存用のパス

# モデルパスをKAGGLE環境に応じて設定します。
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'  # 'lsys-models'を含むディレクトリを見つけてそのパスを設定

# MODEL_PATHを表示します。
print(MODEL_PATH)

# コードのパスを設定します。KAGGLE環境によって異なります。
CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'  # コードパス
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # 保存パス
```

</div>
</details>

In [ ]:
# 入力、モデル、ログ用のパスを設定します。
INPUT_PATH = '/kaggle/input/'  # Kaggleの入力データパス
MODEL_PATH = '/workspace/models/'  # モデル保存用のパス
LOGITS_PATH = '/workspace/logits/'  # ログ保存用のパス

# モデルパスをKAGGLE環境に応じて設定します。
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'  # 'lsys-models'を含むディレクトリを見つけてそのパスを設定

# MODEL_PATHを表示します。
print(MODEL_PATH)

# コードのパスを設定します。KAGGLE環境によって異なります。
CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'  # コードパス
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # 保存パス

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーの並列処理を無効にします。
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # 複数のトークナイザーの並列処理を無効にして、競合を防ぎます。
```

</div>
</details>

In [ ]:
# トークナイザーの並列処理を無効にします。
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # 複数のトークナイザーの並列処理を無効にして、競合を防ぎます。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv(open(DATA + 'train.csv', 'r'))
test = pd.read_csv(open(DATA + 'test.csv', 'r'))
sample = pd.read_csv(DATA + 'sample_submission.csv')

print(len(train), len(test))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 訓練データ、テストデータ、サンプル提出ファイルをCSVから読み込みます。
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # 訓練データを読み込み
test = pd.read_csv(open(DATA + 'test.csv', 'r'))  # テストデータを読み込み
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込み

# 訓練データとテストデータのサンプル数を表示します。
print(len(train), len(test))  # 訓練データとテストデータの行数を表示
```

</div>
</details>

In [ ]:
# 訓練データ、テストデータ、サンプル提出ファイルをCSVから読み込みます。
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # 訓練データを読み込み
test = pd.read_csv(open(DATA + 'test.csv', 'r'))  # テストデータを読み込み
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込み

# 訓練データとテストデータのサンプル数を表示します。
print(len(train), len(test))  # 訓練データとテストデータの行数を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
params = {}
if False:#len(test) < 10: 
    pass;
    params['subsample'] = 30
else:
    # params['subsample'] = 2
    params['fold'] = -1


params['n_epochs'] = 1
params['n_lgb'] = 1
params['model'] = 'microsoft/deberta-v3-small'
```

</div>
<div class="column-right">

# 日本語訳

```python
# パラメータの辞書を初期化します。
params = {}
if False:#len(test) < 10: 
    pass;  # 条件がFalseの場合は何もしない
    params['subsample'] = 30  # サブサンプル数を30に設定
else:
    # サブサンプル数を設定しない
    params['fold'] = -1  # フォールドの設定を-1にする

# その他のモデルに関するパラメータを設定します。
params['n_epochs'] = 1  # エポック数を1に設定
params['n_lgb'] = 1  # LightGBMのモデル数を1に設定
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルをDeBERTaに設定
```

</div>
</details>

In [ ]:
# パラメータの辞書を初期化します。
params = {}
if False:#len(test) < 10: 
    pass;  # 条件がFalseの場合は何もしない
    params['subsample'] = 30  # サブサンプル数を30に設定
else:
    # サブサンプル数を設定しない
    params['fold'] = -1  # フォールドの設定を-1にする

# その他のモデルに関するパラメータを設定します。
params['n_epochs'] = 1  # エポック数を1に設定
params['n_lgb'] = 1  # LightGBMのモデル数を1に設定
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルをDeBERTaに設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# params = {}
FULL = params.get('fold', 0) < 0
N_FOLDS = int(params.get('n_folds', 3)); 
FOLD = int(params.get('fold', 0))
SEED = int(params.get('seed', 3))
SS = int(params.get('subsample', 1))

print(N_FOLDS, FOLD, SEED, SS)

```

</div>
<div class="column-right">

# 日本語訳

```python
# フォールド数、シード値、サブサンプルサイズを設定します。
FULL = params.get('fold', 0) < 0  # フォールドが-1未満の場合はFULLをTrueに設定
N_FOLDS = int(params.get('n_folds', 3))  # フォールドの数を取得、デフォルトは3
FOLD = int(params.get('fold', 0))  # 現在のフォールドを取得、デフォルトは0
SEED = int(params.get('seed', 3))  # シードの値を取得、デフォルトは3
SS = int(params.get('subsample', 1))  # サブサンプル数を取得、デフォルトは1

# 設定されたフォールド数、現在のフォールド、シード、サブサンプルサイズを表示します。
print(N_FOLDS, FOLD, SEED, SS)  # 各設定値を出力
```

</div>
</details>

In [ ]:
# フォールド数、シード値、サブサンプルサイズを設定します。
FULL = params.get('fold', 0) < 0  # フォールドが-1未満の場合はFULLをTrueに設定
N_FOLDS = int(params.get('n_folds', 3))  # フォールドの数を取得、デフォルトは3
FOLD = int(params.get('fold', 0))  # 現在のフォールドを取得、デフォルトは0
SEED = int(params.get('seed', 3))  # シードの値を取得、デフォルトは3
SS = int(params.get('subsample', 1))  # サブサンプル数を取得、デフォルトは1

# 設定されたフォールド数、現在のフォールド、シード、サブサンプルサイズを表示します。
print(N_FOLDS, FOLD, SEED, SS)  # 各設定値を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from sklearn.model_selection import StratifiedKFold

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
if SS > 1: train_ids, test_ids = train_ids[::SS], test_ids[::SS]

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set() 
```

</div>
<div class="column-right">

# 日本語訳

```python
from sklearn.model_selection import StratifiedKFold  # StratifiedKFoldをインポート

# フォールドを取得するための関数を定義します。
def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state=SEED, shuffle=True)  # StratifiedKFoldを使用してフォールドを生成
                .split(X=np.zeros(len(train)), y=train.iloc[:, -3:].idxmax(1)))  # ラベルのインデックスを用いて分割

# 現在のフォールドに応じて訓練データとテストデータのIDを取得します。
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# サブサンプル数が1より大きい場合、IDをサブサンプリングします。
if SS > 1: 
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

# 訓練データとテストデータのIDの数を表示し、重複していないことを確認します。
print(len(train_ids), len(test_ids))  # 訓練IDとテストIDの数を表示
assert set(train_ids) & set(test_ids) == set()  # 訓練とテストIDに重複がないことを確認
```

</div>
</details>

In [ ]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFoldをインポート

# フォールドを取得するための関数を定義します。
def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state=SEED, shuffle=True)  # StratifiedKFoldを使用してフォールドを生成
                .split(X=np.zeros(len(train)), y=train.iloc[:, -3:].idxmax(1)))  # ラベルのインデックスを用いて分割

# 現在のフォールドに応じて訓練データとテストデータのIDを取得します。
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# サブサンプル数が1より大きい場合、IDをサブサンプリングします。
if SS > 1: 
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

# 訓練データとテストデータのIDの数を表示し、重複していないことを確認します。
print(len(train_ids), len(test_ids))  # 訓練IDとテストIDの数を表示
assert set(train_ids) & set(test_ids) == set()  # 訓練とテストIDに重複がないことを確認

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def join_strings(x, ):
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x
    return x
```

</div>
<div class="column-right">

# 日本語訳

```python
# リスト内の文字列を結合するための関数を定義します。
def join_strings(x):
    # xがリストである場合、要素を結合します。Noneの場合は空文字に置き換えます。
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x  # リストをスペースで結合
    return x  # 結合した文字列を返します
```

</div>
</details>

In [ ]:
# リスト内の文字列を結合するための関数を定義します。
def join_strings(x):
    # xがリストである場合、要素を結合します。Noneの場合は空文字に置き換えます。
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x  # リストをスペースで結合
    return x  # 結合した文字列を返します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def len_join_strings(x, ):
    return len(join_strings(x).split())
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結合された文字列の長さを返す関数を定義します。
def len_join_strings(x):
    # join_strings関数を使用して文字列を結合し、その結果をスペースで分割して単語数を返します。
    return len(join_strings(x).split())  # 結合した文字列の単語数を返す
```

</div>
</details>

In [ ]:
# 結合された文字列の長さを返す関数を定義します。
def len_join_strings(x):
    # join_strings関数を使用して文字列を結合し、その結果をスペースで分割して単語数を返します。
    return len(join_strings(x).split())  # 結合した文字列の単語数を返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def len_join_strings_j(x):
    x = json.loads(x)
    return len_join_strings(x)
```

</div>
<div class="column-right">

# 日本語訳

```python
# JSON形式の文字列を解析し、結合された文字列の長さを返す関数を定義します。
def len_join_strings_j(x):
    x = json.loads(x)  # JSON文字列をPythonオブジェクトに変換
    return len_join_strings(x)  # 結合された文字列の長さを返す
```

</div>
</details>

In [ ]:
# JSON形式の文字列を解析し、結合された文字列の長さを返す関数を定義します。
def len_join_strings_j(x):
    x = json.loads(x)  # JSON文字列をPythonオブジェクトに変換
    return len_join_strings(x)  # 結合された文字列の長さを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)
```

</div>
<div class="column-right">

# 日本語訳

```python
# シードを設定して再現性を確保します。
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchのシードを設定
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールのシードを設定
np.random.seed(datetime.datetime.now().microsecond)  # NumPyのシードを設定
```

</div>
</details>

In [ ]:
# シードを設定して再現性を確保します。
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchのシードを設定
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールのシードを設定
np.random.seed(datetime.datetime.now().microsecond)  # NumPyのシードを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# TRAIN = True and not KAGGLE
TRAIN = False
INFER = True # or KAGGLE 
SAVE = False
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習モードや推論モード、保存モードを設定します。
# TRAINはTrueかつKAGGLEでない場合に学習を行うかを示します。
TRAIN = False  # 学習を行うかどうかを設定（現在はFalse）
INFER = True  # 推論を行うかどうかを設定（現在はTrue、KAGGLE環境でも使用可能）
SAVE = False  # モデルや結果を保存するかどうかを設定（現在はFalse）
```

</div>
</details>

In [ ]:
# 学習モードや推論モード、保存モードを設定します。
# TRAINはTrueかつKAGGLEでない場合に学習を行うかを示します。
TRAIN = False  # 学習を行うかどうかを設定（現在はFalse）
INFER = True  # 推論を行うかどうかを設定（現在はTrue、KAGGLE環境でも使用可能）
SAVE = False  # モデルや結果を保存するかどうかを設定（現在はFalse）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMとカウントベクトライザーをインポートします。
import lightgbm as lgb  # LightGBMライブラリをインポート
from sklearn.feature_extraction.text import CountVectorizer  # テキストデータの特徴量抽出に使用するカウントベクトライザーをインポート
```

</div>
</details>

In [ ]:
# LightGBMとカウントベクトライザーをインポートします。
import lightgbm as lgb  # LightGBMライブラリをインポート
from sklearn.feature_extraction.text import CountVectorizer  # テキストデータの特徴量抽出に使用するカウントベクトライザーをインポート

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
LGB = True
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0
INFER_LGB = not TRAIN and LGB
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMを使用するかどうかを設定します。
LGB = True  # LightGBMを使用するかどうかのフラグを設定
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # 学習モードでLightGBMを使用するかのフラグ
INFER_LGB = not TRAIN and LGB  # 学習していない場合にLightGBMによる推論を行うかのフラグ
```

</div>
</details>

In [ ]:
# LightGBMを使用するかどうかを設定します。
LGB = True  # LightGBMを使用するかどうかのフラグを設定
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # 学習モードでLightGBMを使用するかのフラグ
INFER_LGB = not TRAIN and LGB  # 学習していない場合にLightGBMによる推論を行うかのフラグ

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 事前に保存したカウントベクトライザーを読み込みます。
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # cvecモデルを読み込む
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # ccvecモデルを読み込む
```

</div>
</details>

In [ ]:
# 事前に保存したカウントベクトライザーを読み込みます。
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # cvecモデルを読み込む
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # ccvecモデルを読み込む

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def symlog(x): return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())
    x = symlog(x)
    return x

def get_features(df):
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    v = np.hstack([
    # pfeat, 
          afeat - bfeat, np.abs(afeat - bfeat), 
    # afeat + bfeat
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except: pass

    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)
            
    extras.append(df[c].str.len())
    extras.append(df[c].str.split().apply(lambda x: len(x)))
    
    extras = np.stack(extras, axis = 1)
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])
    return np.hstack([v, extras])
    # return v

```

</div>
<div class="column-right">

# 日本語訳

```python
# シンメトリック対数変換を行う関数を定義します。
def symlog(x): 
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # 対数変換と符号付き変換を行う

# 密な行列を変換する関数を定義します。
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 行列を密な形式に変換し、float32型にする
    x = symlog(x)  # シンメトリック対数変換を適用
    return x  # 変換した値を返す

# 特徴量を取得するための関数を定義します。
def get_features(df):
    # プロンプトに対する特徴を取得
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])  # cvecとccvecを用いてtransform

    # モデルAに対する特徴を取得
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ] 
                    for v in [cvec, ccvec]
                ])  # response_aに対してtransform

    # モデルBに対する特徴を取得
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ] 
                    for v in [cvec, ccvec]
                ])  # response_bに対してtransform
    
    # 特徴量を計算
    v = np.hstack([
    # pfeat, 
          afeat - bfeat, np.abs(afeat - bfeat), 
    # afeat + bfeat
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # 投票モデルが存在する場合、特徴量を正規化
    except: 
        pass  # エラーが発生しても何もしない

    # 追加の特徴量を計算
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # 確認したい追加の文字列
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # それぞれのカラムで指定した文字列の出現回数をカウント
            
    extras.append(df[c].str.len())  # 各カラムの文字列長を追加
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 各カラムの単語数を取得及び追加
    
    extras = np.stack(extras, axis=1)  # 追加特徴量をスタック
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 特徴量を根号と対数変換

    return np.hstack([v, extras])  # すべての特徴量を結合して返す
    # return v  # vだけを返す選択肢もある
```

</div>
</details>

In [ ]:
# シンメトリック対数変換を行う関数を定義します。
def symlog(x): 
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # 対数変換と符号付き変換を行う

# 密な行列を変換する関数を定義します。
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 行列を密な形式に変換し、float32型にする
    x = symlog(x)  # シンメトリック対数変換を適用
    return x  # 変換した値を返す

# 特徴量を取得するための関数を定義します。
def get_features(df):
    # プロンプトに対する特徴を取得
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])  # cvecとccvecを用いてtransform

    # モデルAに対する特徴を取得
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ] 
                    for v in [cvec, ccvec]
                ])  # response_aに対してtransform

    # モデルBに対する特徴を取得
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ] 
                    for v in [cvec, ccvec]
                ])  # response_bに対してtransform
    
    # 特徴量を計算
    v = np.hstack([
    # pfeat, 
          afeat - bfeat, np.abs(afeat - bfeat), 
    # afeat + bfeat
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # 投票モデルが存在する場合、特徴量を正規化
    except: 
        pass  # エラーが発生しても何もしない

    # 追加の特徴量を計算
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # 確認したい追加の文字列
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # それぞれのカラムで指定した文字列の出現回数をカウント
            
    extras.append(df[c].str.len())  # 各カラムの文字列長を追加
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 各カラムの単語数を取得及び追加
    
    extras = np.stack(extras, axis=1)  # 追加特徴量をスタック
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 特徴量を根号と対数変換

    return np.hstack([v, extras])  # すべての特徴量を結合して返す
    # return v  # vだけを返す選択肢もある

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 事前に保存したLightGBMモデルを読み込みます。
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込む
```

</div>
</details>

In [ ]:
# 事前に保存したLightGBMモデルを読み込みます。
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込む

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if INFER and params.get('n_lgb', 1) > 0:
    df = test
    yps = []; b = 1000
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])
        ypms = []
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))
        yps.append(np.stack(ypms).mean(0))
        # break;
        print('.', end = '')
        
        if len(yps) % 2 == 0:
            gc.collect()
    print()

    yp = np.concatenate(yps)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を行うための処理を実行します。
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータをdfに設定
    yps = []  # モデルの予測結果を格納するリストを初期化
    b = 1000  # バッチサイズを設定
    
    # テストデータをバッチ処理で繰り返す
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得
        ypms = []  # 個々のモデルの予測結果を格納するリストを初期化
        
        # 各LightGBMモデルについて推論を実施
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))  # 各モデルの予測確率を追加
        
        # 平均を取った確率をypsに追加
        yps.append(np.stack(ypms).mean(0))  # 各モデルの予測をスタックし、平均を計算
        # break;  # デバッグのためにループを中断する行（コメントアウト）

        print('.', end='')  # 進行状況の表示
        
        # ypsの長さが偶数のときにガベージコレクションを実行
        if len(yps) % 2 == 0:
            gc.collect()  # メモリを整理
    print()  # 改行

    # すべての予測結果を結合
    yp = np.concatenate(yps)  # 予測確率を一つの配列に結合
```

</div>
</details>

In [ ]:
# 推論を行うための処理を実行します。
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータをdfに設定
    yps = []  # モデルの予測結果を格納するリストを初期化
    b = 1000  # バッチサイズを設定
    
    # テストデータをバッチ処理で繰り返す
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得
        ypms = []  # 個々のモデルの予測結果を格納するリストを初期化
        
        # 各LightGBMモデルについて推論を実施
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))  # 各モデルの予測確率を追加
        
        # 平均を取った確率をypsに追加
        yps.append(np.stack(ypms).mean(0))  # 各モデルの予測をスタックし、平均を計算
        # break;  # デバッグのためにループを中断する行（コメントアウト）

        print('.', end='')  # 進行状況の表示
        
        # ypsの長さが偶数のときにガベージコレクションを実行
        if len(yps) % 2 == 0:
            gc.collect()  # メモリを整理
    print()  # 改行

    # すべての予測結果を結合
    yp = np.concatenate(yps)  # 予測確率を一つの配列に結合

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_preds = yp
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMモデルからの予測結果をlgb_predsに格納します。
lgb_preds = yp  # 推論結果をlgb_predsに設定
```

</div>
</details>

In [ ]:
# LightGBMモデルからの予測結果をlgb_predsに格納します。
lgb_preds = yp  # 推論結果をlgb_predsに設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Blend predictions

$\operatorname{preds} = 0.2 \cdot \operatorname{lgbm boosting preds} + 0.8 \cdot \operatorname{llama preds}$


</div>
<div class="column-right">

# 日本語訳

## 予測のブレンド

$\operatorname{preds} = 0.2 \cdot \operatorname{LGBMブースティング予測} + 0.8 \cdot \operatorname{ラマ予測}$

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_wt = 0.2 
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds
```

</div>
<div class="column-right">

# 日本語訳

```python
# LGBMモデルとラマモデルの予測をブレンドします。
lgb_wt = 0.2  # LGBMモデルの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # ブレンドした予測結果を計算
```

</div>
</details>

In [ ]:
# LGBMモデルとラマモデルの予測をブレンドします。
lgb_wt = 0.2  # LGBMモデルの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # ブレンドした予測結果を計算

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out = pd.DataFrame(preds, 
                index = df.id, 
                    columns = train.columns[-3:])
display(out.head())
```

</div>
<div class="column-right">

# 日本語訳

```python
# ブレンドした予測結果をデータフレームに変換します。
out = pd.DataFrame(preds, 
                index=df.id,  # データフレームのインデックスとしてIDを設定
                columns=train.columns[-3:])  # カラム名として訓練データの最後の3カラムを設定
display(out.head())  # データフレームの先頭5行を表示
```

</div>
</details>

In [ ]:
# ブレンドした予測結果をデータフレームに変換します。
out = pd.DataFrame(preds, 
                index=df.id,  # データフレームのインデックスとしてIDを設定
                columns=train.columns[-3:])  # カラム名として訓練データの最後の3カラムを設定
display(out.head())  # データフレームの先頭5行を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out.to_csv('submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果をCSVファイルとして保存します。
out.to_csv('submission.csv')  # 予測結果を'submission.csv'という名前で保存
```

</div>
</details>

In [ ]:
# 予測結果をCSVファイルとして保存します。
out.to_csv('submission.csv')  # 予測結果を'submission.csv'という名前で保存